0.0 CARGA DE BIBLIOTECAS, VARIABLES, CLASES Y MÉTODOS IMPORTANTES

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv1D, Dropout, LSTM, Dense
import pickle
import json
import os

archivo_json = '../../Chessia_Data_Convert/Target Data/Datos-V2.json'
tokens_lilith03 = "tokens03.pkl"

class Partida:
    def __init__(self):
        self.movimientos = []
        self.resultado = None

# Función para cargar partidas en lotes
def cargar_partidas_por_rango(archivo_json, inicio=0, fin=103350):
    total_movimientos = []
    total_resultados = []
    with open(archivo_json, 'r') as file:
        # Carga el contenido completo como una lista de objetos JSON
        lista_json = json.load(file)
        
        # Itera sobre cada objeto JSON en la lista
        for i in range(inicio, fin):
            total_movimientos.append(lista_json[i]['partida']['movimientos'])
            total_resultados.append(lista_json[i]['partida']['resultado'])
            
    return total_movimientos, total_resultados
        

3.0 PROCESADO DE DATOS DE LILITH03 - USA ProcesadorPGN_V2

In [4]:
# Convertir el generador en una lista y extraer la primera tupla
recogido = cargar_partidas_por_rango(archivo_json, fin=100000)
movimientos = recogido[0]
resultados = recogido[1]
# Crear y ajustar el tokenizador
tokenizer = Tokenizer()
tokenizer.fit_on_texts(movimientos)

#Borramos los antiguos tokens si los hubiese
if os.path.exists(tokens_lilith03):
    os.remove(tokens_lilith03)

# Guardar el objeto Tokenizer en un archivo usando pickle
with open(tokens_lilith03, "wb") as f:
    pickle.dump(tokenizer, f)

# Convertir todas las jugadas a secuencias de números enteros y luego a tensores válidos
movimientos_tokenized = tokenizer.texts_to_sequences(movimientos)
movimientos_tokenized = pad_sequences(movimientos_tokenized, maxlen=500, padding="pre")

resultados_tokenized = tf.one_hot([int(resultado) for resultado in resultados], depth=3)  # 3 clases: empate, victoria blanca, victoria negra

tensor_mov_entrena = tf.constant(movimientos_tokenized[:int(len(movimientos_tokenized) * 0.7) ])
tensor_res_entrena = tf.constant(resultados_tokenized [:int(len(resultados_tokenized)  * 0.7) ])
tensor_mov_prueba =  tf.constant(movimientos_tokenized[ int(len(movimientos_tokenized) * 0.7):])
tensor_res_prueba =  tf.constant(resultados_tokenized [ int(len(resultados_tokenized)  * 0.7):])

<u>3.1 LILITH 03 - DEFINICIÓN, ENTRENAMIENTO Y GRÁFICAS DE DATOS CON POCOS VALORES</u>

        3.1.1 Creación del modelo

In [12]:
# 1. Definir el modelo
modelo = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=200),
    LSTM(units=256, return_sequences=True),
    Dropout(0.3),
    LSTM(units=256),
    Dropout(0.3),
    Dense(units=128, activation='relu'),
    Dropout(0.3),
    Dense(units=3, activation='softmax')
])

# 2. Compilar el modelo
# optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001) 
modelo.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Definir EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)

        3.1.2 Entrenamiento

In [14]:
# Entrenar el modelo y guardar el historial del entrenamiento
history = modelo.fit(
    tensor_mov_entrena,
    tensor_res_entrena,
    epochs=20,
    batch_size=256,
    validation_data=(tensor_mov_prueba, tensor_res_prueba),
    callbacks=[early_stopping]
)

modelo.save("modelos/lilith03.keras")

print("\n\nLoss en los datos de prueba:",    modelo.evaluate(tensor_mov_prueba, tensor_res_prueba)[0])
print("Precisión en los datos de prueba:", modelo.evaluate(tensor_mov_prueba, tensor_res_prueba[1]))

Epoch 1/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 1298s 5s/step - accuracy: 0.6095 - loss: 0.8663 - val_accuracy: 0.6300 - val_loss: 0.8379
Epoch 2/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 1271s 5s/step - accuracy: 0.6596 - loss: 0.7759 - val_accuracy: 0.7405 - val_loss: 0.6030
Epoch 3/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 1286s 5s/step - accuracy: 0.7603 - loss: 0.5758 - val_accuracy: 0.7596 - val_loss: 0.5717
Epoch 4/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 1288s 5s/step - accuracy: 0.7877 - loss: 0.5226 - val_accuracy: 0.7732 - val_loss: 0.5466
Epoch 5/20
274/274 ━━━━━━━━━━━━━━━━━━━━ 1309s 5s/step - accuracy: 0.8083 - loss: 0.4775 - val_accuracy: 0.7833 - val_loss: 0.5321
Epoch 6/20
268/274 ━━━━━━━━━━━━━━━━━━━━ 25s 4s/step - accuracy: 0.8200 - loss: 0.4526

        3.1.3 ESTADÍSTICAS Y GRÁFICAS

In [ ]:
epocas = range(1, len(history.history['loss']) + 1)

# Graficar la precisión
plt.plot(epocas, history.history['accuracy'], 'r', label='Precisión en entrenamiento')
plt.plot(epocas, history.history['val_accuracy'], 'b', label='Precisión en validación')
plt.title('Precisión en entrenamiento y validación')
plt.xlabel('Épocas')
plt.ylabel('Precisión')
plt.legend()
plt.show()

# Graficar la pérdida
# plt.plot(epocas, history.history['loss'], 'r', label='Pérdida en entrenamiento')
# plt.plot(epocas, history.history['val_loss'], 'b', label='Pérdida en validación')
# plt.title('Pérdida en entrenamiento y validación')
# plt.xlabel('Épocas')
# plt.ylabel('Pérdida')
# plt.legend()
# plt.show()

2.2 PRUEBA DE PREDICCIÓN LILITH03

    (bastante mal, además los tokens no se recogen del fichero no sé por qué)

        2.2.1 RECOGIDA DE DATOS Y TOKENS

In [13]:
# Convertir el generador en una lista y extraer la primera tupla
recogido = next(cargar_partidas_por_lotes(archivo_json, tam_lote=100070))
movimientos = recogido[0]
resultados = recogido[1]

# Como la carga desde el archo funciona mal, los vuelvo a generar y rezo para que sean iguales
# Crear y ajustar el tokenizador
nuevoTokenizer = Tokenizer()
nuevoTokenizer.fit_on_texts(movimientos[:100000])

movimientos = recogido[0][100040:]
resultados = recogido[1][100040:]


        2.2.2 PREDICCIÓN Y MUESTRA DE DATOS

            (problemita de versiones, pero vamos, que tampoco predecía)

In [14]:
np.set_printoptions(suppress=True, precision=4)

lilith02 = tf.keras.models.load_model("modelos/lilith02.h5")

# Crear y ajustar el tokenizador
nuevoTokenizer.fit_on_texts(movimientos)

# Convertir las jugadas a secuencias de números enteros
movimientos_secuencias_predict = nuevoTokenizer.texts_to_sequences(movimientos)

movimientos_prueba = []
for i in range(len(movimientos_secuencias_predict[0])):
    if movimientos_secuencias_predict[0][i] != 1:
        movimientos_prueba.append(movimientos_secuencias_predict[0][i])

resultados_predichos = lilith02.predict(movimientos_secuencias_predict)

for i in range(len(resultados_predichos)):
    print(f"Predicción: {resultados_predichos[i]} -- Dice: {np.argmax(resultados_predichos[i])} -- Real: {resultados[i]}")

c:\Users\arube\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
c:\Users\arube\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\embedding.py:89: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


ValueError: Could not interpret initializer identifier: {'module': 'keras.initializers', 'class_name': 'Orthogonal', 'config': {'gain': 1.0, 'seed': None}, 'registered_name': None}